In [2]:
import os
import pandas as pd

def list_csv_files(folder_path):
    return [f for f in os.listdir(folder_path) if f.endswith('.csv') and f != "merged_output.csv"]

def process_and_merge_csv(file_path, selected_columns, output_df, merge_column):
    try:
        filename = os.path.splitext(os.path.basename(file_path))[0]  
        
        # Read file
        chunk = pd.read_csv(file_path, encoding="utf-8", encoding_errors="replace", sep=";", dtype=str)
        chunk.columns = [col.strip() for col in chunk.columns]  
        chunk = chunk[selected_columns]  
        
        # Rename columns (except merge column)
        chunk = chunk.rename(columns={col: f"{filename}_{col}" for col in selected_columns if col != merge_column})
        
        # Merge with existing data
        if output_df is None:
            output_df = chunk
        else:
            output_df = pd.merge(output_df, chunk, on=merge_column, how="outer")  # Merge on common column
            
        return output_df
    except Exception as e:
        print(f"⚠️ Error processing '{file_path}': {e}")
        return output_df

def get_column_selection(file_path):
    try:
        sample_df = pd.read_csv(file_path, encoding="utf-8", encoding_errors="replace", sep=";", dtype=str, nrows=100)
        sample_df.columns = [col.strip() for col in sample_df.columns]
    except Exception as e:
        print(f"⚠️ Error reading sample rows from '{file_path}': {e}")
        return None

    print(f"\nColumns in '{file_path}': {list(sample_df.columns)}")
    
    while True:
        selected = input("Select columns to keep (comma-separated): ").split(',')
        selected = [col.strip() for col in selected]

        invalid_columns = [col for col in selected if col not in sample_df.columns]
        if invalid_columns:
            print(f"\n⚠️ These columns do not exist in '{file_path}': {invalid_columns}. Please enter valid column names.")
        else:
            break 

    return selected

def merge_large_csvs(folder_path, csv_files, output_file):
    global merge_column
    merge_column = None
    merged_df = None  

    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        selected_columns = get_column_selection(file_path)
        if selected_columns is None:
            continue

        if merge_column is None:
            print(f"\nAvailable columns for merging: {selected_columns}")
            while True:
                merge_column = input("\nEnter the column name to merge on: ").strip()
                if merge_column in selected_columns:
                    break
                print(f"\n⚠️ '{merge_column}' not found in selected columns. Please enter a valid column name.")

        # Ensure merge column is included
        if merge_column not in selected_columns:
            selected_columns.insert(0, merge_column)

        # Process and merge data
        merged_df = process_and_merge_csv(file_path, selected_columns, merged_df, merge_column)

    # Save merged data to CSV
    if merged_df is not None:
        merged_df.to_csv(output_file, index=False)
    return merge_column

def generate_correlation_matrix(output_file):
    """Generate a correlation matrix for numerical columns."""
    try:
        df = pd.read_csv(output_file, encoding="utf-8", encoding_errors="replace", dtype=str)
        df = df.apply(pd.to_numeric, errors='coerce')  
        correlation_matrix = df.corr()
        
        correlation_output_file = output_file.replace(".csv", "_correlation_matrix.csv")
        correlation_matrix.to_csv(correlation_output_file)
        print(f"\n✅ Correlation matrix saved as '{correlation_output_file}'")
    except Exception as e:
        print(f"⚠️ Error generating correlation matrix: {e}")

def main():
    folder_path = input("Enter the folder path containing CSV files: ").strip()
    csv_files = list_csv_files(folder_path)
    
    if not csv_files:
        print("⚠️ No CSV files found in the specified folder.")
        return
    
    print(f"\n✅ Found {len(csv_files)} CSV files.")

    output_file = os.path.join(folder_path, "merged_output.csv")
    
    # Remove old output file if exists
    if os.path.exists(output_file):
        os.remove(output_file)

    merge_column = merge_large_csvs(folder_path, csv_files, output_file)

    if merge_column is not None:
        print(f"\n✅ Merged CSV saved as '{output_file}' (Merged on '{merge_column}')")
        generate_correlation_matrix(output_file)
    else:
        print("\n❌ Merging failed. No output file generated.")

if __name__ == "__main__":
    main()

Enter the folder path containing CSV files:  /Users/bharat_puri/Documents/AquaAware/Copy



✅ Found 2 CSV files.

Columns in '/Users/bharat_puri/Documents/AquaAware/Copy/Manganese.csv': ['GEMS.Station.Number', 'Sample.Date', 'Sample.Time', 'Depth', 'Parameter.Code', 'Analysis.Method.Code', 'Value.Flags', 'Value', 'Unit', 'Data.Quality']


Select columns to keep (comma-separated):  GEMS.Station.Number, Value



Available columns for merging: ['GEMS.Station.Number', 'Value']



Enter the column name to merge on:  GEMS.Station.Number



Columns in '/Users/bharat_puri/Documents/AquaAware/Copy/Carbon.csv': ['GEMS.Station.Number', 'Sample.Date', 'Sample.Time', 'Depth', 'Parameter.Code', 'Analysis.Method.Code', 'Value.Flags', 'Value', 'Unit', 'Data.Quality']


Select columns to keep (comma-separated):  GEMS.Station.Number, Value



✅ Merged CSV saved as '/Users/bharat_puri/Documents/AquaAware/Copy/merged_output.csv' (Merged on 'GEMS.Station.Number')

✅ Correlation matrix saved as '/Users/bharat_puri/Documents/AquaAware/Copy/merged_output_correlation_matrix.csv'


In [4]:

file_path = "/Users/bharat_puri/Documents/AquaAware/Copy/merged_output.csv"

# Read only the first 10 rows
df = pd.read_csv(file_path, nrows=10)

print(df)

  GEMS.Station.Number  Manganese_Value  Carbon_Value
0            ARG00014            0.007          5.28
1            ARG00014            0.224          5.28
2            ARG00014            0.002          5.28
3            ARG00014            1.920          5.28
4            ARG00014            0.043          5.28
5            ARG00014            0.563          5.28
6            ARG00014            0.006          5.28
7            ARG00014            0.213          5.28
8            ARG00014            0.006          5.28
9            ARG00014            3.925          5.28
